### [1]

### Importing the libraries:

In [2]:
import pandas as pd
import numpy as np
import json
import shutup 
from datetime import datetime
shutup.please()

##### Description:

As a standard, I am importing the pandas, numpy, json and shutup libraries which are required for my computation purposes. 

Since this is the feature engineering part, I am using datetime as my primary variable to measure the transactions for the multiswipe. I am using a simple method of assumption for the reversal, making it easier to find the reversal transactions. 
##### Libraries used for Feature Engineering and Data Wrangling: 

    datetime
    pandas

##### Attributes and functions used for data wrangling:
    Datetime objects
    Lambda functions for conversions
    Group by
    For loops and conditional statements

### [2]

### Reading the file:

In [3]:
df = pd.read_json("transactions.txt", lines = True)

In [4]:
pd.set_option('display.max_columns', None)

#### Refer to [2.1] from 'Question 1.ipynb' for additional information about reading the file.

### [3]

### Data Wrangling:

#### [3.1]

#### Assumptions for the reversal and multiswipe transactions:

**Assumption:** 

From a casual look at the dataset, I can assertively get the number of reversals by just filtering the number of reversal transaction type.

This is not a huge task if you think about it, although there are other things that one should consider. And one such thing is provided in the question description.

The questions to look at the transactions that are duplicate and followed by a purchase. This means that there are either transactions that are reversed or are multiswipe.

Now using the conditional operators, I can find the reversal transactions that are a purchase followed by a reversal. 

**Method:**

Assuming the REVERSED transactions are purchases that are always followed by a reverse, I am trying to get the transactions whose transaction amounts are same, else I would consider them as a refund as opposed to a reversal transactions. 

Here I am not calculating the amount of transactions yet, because I want to be sure about the amount between the times the reversal is going to happen that I will be discussing the next few steps.

In [5]:
count=0
for i in range(1,len(df)):
    if(df.transactionAmount[i]==df.transactionAmount[i-1] and df.transactionType[i-1]=="PURCHASE" and df.transactionType[i]=="REVERSAL"):
        count+=1
count

6064

##### Description:

The above function describes the transactions that are going to be reversed where the transactions are duplicated, and the purchases are followed by a reversal

##### Input Parameters:

We are using the following the transactionAmount column to find the immediate duplicate transactions, and checking whether the purchases are followed by the reversal. 

##### Output:

I am outputting the number of reversal transactions. This will be further checked with the number of minutes. 

In [6]:
len(df[df.transactionType=="REVERSAL"])

20303

Since there are more number of attributes that say they are reversal, I assume that a lot of those could be refunds or fraudulent transactions.

### [4]

### Feature Engineering

#### Method:
    
***Why am I doing feature engineering while doing the question?***

Since I am going to use the minutes as a feature, I would like to remove a few things before hand and store the final features into the dataframe. This will help me save time on the model and not do excessive stuff other than training and testing the model.


#### [4.1] 


##### Dropping unnecessary columns:

In the Question 1 notebook, we have observed that there are a features that are completely null and will no be useful for our dataset. 

In [7]:
df = df.drop(["echoBuffer", "merchantState", "merchantCity", "merchantZip", "posOnPremises", "recurringAuthInd"], axis = 1)

In [8]:
df.shape

(786363, 23)

#### [4.2] 

##### CVV MATCH

Adding a feature called CVV match so that we can drop the cardCVV and enteredCVV and the model doesn't have to do extra work on correlating the features

In [9]:
df["CVV Match"]  = df.cardCVV == df.enteredCVV

In [10]:
df["CVV Match"].value_counts()

True     779348
False      7015
Name: CVV Match, dtype: int64

#### [4.3] 

##### Dropping more columns to make a new dataframe

##### One last check for the df and how many records can be dropped from the dataset using the head and co-relation matrix:

In [11]:
df.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,CVV Match
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,01,rideshare,06/2023,2015-03-14,2015-03-14,414,414,1803,PURCHASE,0.0,False,False,False,True
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,01,entertainment,02/2024,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.0,True,False,False,True
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.0,False,False,False,True
3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.0,False,False,False,True
4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,01,fastfood,10/2029,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.0,True,False,False,True


In [12]:
df.corr()

,accountNumber,customerId,creditLimit,availableMoney,transactionAmount,cardCVV,enteredCVV,cardLast4Digits,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,CVV Match
accountNumber,1.000000,1.000000,0.140673,0.066345,-0.001364,-0.011432,-0.011376,0.038517,0.162248,0.006099,-0.001057,-0.004011,0.004877
customerId,1.000000,1.000000,0.140673,0.066345,-0.001364,-0.011432,-0.011376,0.038517,0.162248,0.006099,-0.001057,-0.004011,0.004877
creditLimit,0.140673,0.140673,1.000000,0.834977,0.005581,0.051006,0.051088,0.125611,0.653652,-0.003929,0.002391,0.003108,0.001343
availableMoney,0.066345,0.066345,0.834977,1.000000,-0.010070,0.016253,0.016323,0.073879,0.129332,0.002508,0.004067,-0.001538,-0.000583
transactionAmount,-0.001364,-0.001364,0.005581,-0.010070,1.000000,-0.000233,-0.000141,-0.001513,0.023905,0.037361,0.001751,0.075651,0.000701
cardCVV,-0.011432,-0.011432,0.051006,0.016253,-0.000233,1.000000,0.996008,-0.011608,0.069559,-0.045319,-0.000416,0.008652,-0.002996
enteredCVV,-0.011376,-0.011376,0.051088,0.016323,-0.000141,0.996008,1.000000,-0.011485,0.069610,-0.045113,-0.000200,0.008655,0.008429
cardLast4Digits,0.038517,0.038517,0.125611,0.073879,-0.001513,-0.011608,-0.011485,1.000000,0.124745,0.040726,0.002087,0.000886,-0.001305
currentBalance,0.162248,0.162248,0.653652,0.129332,0.023905,0.069559,0.069610,0.124745,1.000000,-0.010529,-0.001285,0.007715,0.003221
cardPresent,0.006099,0.006099,-0.003929,0.002508,0.037361,-0.045319,-0.045113,0.040726,-0.010529,1.000000,0.040393,-0.043418,0.002988


##### Dropping the columns customerId, dateOfAddressChange, AccountOpenDate, customerId, cardCVV, enteredCVV, cardLast4Digits

Creating a new dataframe afterwards called the 'new_df'

In [13]:
new_df = df.drop(["customerId", "currentExpDate", "dateOfLastAddressChange", "accountOpenDate", "cardCVV", "enteredCVV", "cardLast4Digits"], axis = 1)


### [5]


#### Calculating the minutes for finding the multiswipe transactions:



#### [5.1]

#### Making the transaction time into a datetime object as opposed to a string.

Using the lambda functions twice to convert the string into a datetime object.

**Method:**
    
The strings onto the DateTime object are joined using the letter 'T'. We will split it into a list of two strings where the one 'T' contains date and the one after contains time.

The lambda function is used again to join the two strings in the list into a single string. 

In [14]:
new_df.transactionDateTime = new_df.transactionDateTime.apply(lambda x : x.split("T"))
new_df.transactionDateTime = new_df.transactionDateTime.apply(lambda x: ' '.join(x))

##### [5.1.1]

Trying a demo of the transactionDateTime series by outputting the first variable, which will result in a string of the below format. 

In [15]:
x = new_df.transactionDateTime[0]
x

'2016-08-13 14:27:32'

Now using the above format of string and converting the series into a DateTime object series. 

In [16]:
new_df.transactionDateTime = new_df.transactionDateTime.apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

In [18]:
new_df.dtypes

accountNumber                        int64
creditLimit                          int64
availableMoney                     float64
transactionDateTime         datetime64[ns]
transactionAmount                  float64
merchantName                        object
acqCountry                          object
merchantCountryCode                 object
posEntryMode                        object
posConditionCode                    object
merchantCategoryCode                object
transactionType                     object
currentBalance                     float64
cardPresent                           bool
expirationDateKeyInMatch              bool
isFraud                               bool
CVV Match                             bool
dtype: object

#### Here we see that the transactionDateTime is successfully found to be a datetime object

#### [5.2]
#### Using functions to calculate the difference between the transaction Dates:

In [25]:
dates=new_df['transactionDateTime'].tolist()
accntnum=new_df['accountNumber'].unique().tolist()
results=[]
def transactionPeriod(df):
    dates=df.transactionDateTime.tolist()
    result=[x-dates[i-1] for i,x in enumerate(dates) if i>0]
    return result

for i in accntnum:
    sliced_df= new_df[new_df.accountNumber==i]
    results.append(transactionPeriod(sliced_df))
    


map_results={}
for accnt,re in zip(accntnum,results):
    map_results[accnt]=re

##### Description:
Using the above codeblock to calculate the transactionPeriod and mapping the results into a single hashtable/dictionary.

The function itself calculates the difference between the transactions dates and times for each accountnumber from the second transaction. 
First transaction is left aside because it doesn't have anything preceding it.


This entire process is done for a sliced dataframe of all the accountNumbers.

When we have the difference in transactions period, we zip this into a hastable.


##### Input Parameters:

For the fucntion:
    
    A sliced dataframe for each account number.

For the accountnumbers:
    
    Slicing of dataframe according to the accountNumber

For the hastable:
    
    AccountNumber and the difference in datetime
    
    
##### Additional predefined parameters:

    A list of accountNumbers
    A list of transactionalDateTime object

##### Outputs:

A hastable containing the difference of the datetime 

#### [5.3]

#### Calculating the minutes between each transaction for the account numbers:


In [26]:
def calculate_min(delta):
    days,seconds=delta.days,delta.seconds
    hours=days*24 + seconds/3600
    minutes= hours * 60
    return minutes

res_min=[]
for i in map_results.values():
    res_min.append([ calculate_min(j) for j in i])

##### Description:

This function 'calculate_min', helps with finding the minutes of the datetime object. As the datetime.delta object only gives out days and seconds, we used both of them to find the minutes


##### Input Parameters:

We calculate the minutes by passing the previous hastable values into the function.

##### Output Parameters:

A res_min map of minutes between the two transactions


In [27]:
minutes_delta = []
for x in res_min:
    minutes_delta.append(0)
    for i in x:
        minutes_delta.append(i)

##### Description:

The above codeblock helps with minutes into a array input. Since it is a list of arrays, we use the first value of the array as 0 for the first transaction and rest else are appended until the end of each array making it the same length of the dataframe.

##### Input Parameters:

Only the res_min is passed as an input.

##### Output Parameters:

A list of minutes between the two transactions for each accountNumber.


**Assumption:**
    
I assume multiswipe transactions happen in less than 2 minutes.


From the above, it assumes that there are almost 17K Multiswipe transactions. Let's put it into the dataframe and calculate the actual number with another condition.

I used this article to learn more about multiswipe transactions: [Fraud Detection]( https://www.capitalone.com/bank/security-fraud-protection/)

In [28]:
count=0
for x in minutes_delta:
    if(x>0 and x<=2):
        count+=1
count

17370

In [30]:
new_df["multiswipeMinutes"] = minutes_delta

In [31]:
new_df.head()

,accountNumber,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,CVV Match,multiswipeMinutes
0,737265056,5000,5000.0,2016-08-13 14:27:32,98.55,Uber,US,US,02,01,rideshare,PURCHASE,0.0,False,False,False,True,0.000000
1,737265056,5000,5000.0,2016-10-11 05:05:54,74.51,AMC #191138,US,US,09,01,entertainment,PURCHASE,0.0,True,False,False,True,84398.366667
2,737265056,5000,5000.0,2016-11-08 09:18:39,7.47,Play Store,US,US,09,01,mobileapps,PURCHASE,0.0,False,False,False,True,40572.750000
3,737265056,5000,5000.0,2016-12-10 02:14:50,7.47,Play Store,US,US,09,01,mobileapps,PURCHASE,0.0,False,False,False,True,45656.183333
4,830329091,5000,5000.0,2016-03-24 21:04:46,71.18,Tim Hortons #947751,US,US,02,01,fastfood,PURCHASE,0.0,True,False,False,True,0.000000


**Assumption:**
    
None of the multiswipe transactions should be a REVERSAL.

In [33]:
count=0
for i in range(1,len(new_df)):
    if(new_df.transactionAmount[i]==new_df.transactionAmount[i-1] and new_df.transactionType[i-1]=="PURCHASE" and new_df.transactionType[i]=="REVERSAL"):
        count+=1
        print(new_df.multiswipeMinutes[i])
    if(count==10):
        break
count

2.5
22181.68333333333
0.75
2.433333333333333
1.7166666666666668
1.2666666666666666
2.833333333333333
0.5666666666666667
0.3
0.7166666666666667


10

#### This ensures that the reversal may or may not happen within two minutes of time. Therefore, none of the multiswipe transactions should be a reversal

Since there is a difference between the transactionTime, there will not be any duplicated values.

#### [5.5]
#### Multi-swipe

In [36]:
count=0
amount = 0
for i in range(1, len(new_df)):
    if(new_df.multiswipeMinutes[i]>0 and new_df.multiswipeMinutes[i]<=2 and new_df.transactionType[i]!="REVERSAL"):
        count+=1
        amount+=new_df.transactionAmount[i]

##### Description:
    
Using a loop and conditional statements to gather the amount and count of the multiswipe transactions.

In [38]:
print("No. of multiswipe transactions: {}".format(count))
print("Total amount of the multiswipe transactions: {}".format(amount))

No. of multiswipe transactions: 12925
Total amount of the multiswipe transactions: 1878098.2800000052


In [39]:
sum(new_df.transactionAmount)

107720557.52999923

So almost 2% of the entire amount in the dataset are multiswipe transactions

#### [5.6]
#### Reversal

In [41]:
count=0
amount=0
for i in range(1,len(new_df)):
    if(new_df.transactionAmount[i]==new_df.transactionAmount[i-1] and new_df.transactionType[i-1]=="PURCHASE" and new_df.transactionType[i]=="REVERSAL"):
        count+=1
        amount+=new_df.transactionAmount[i]

In [42]:
print("No. of reversed transactions: {}".format(count))
print("Total amount of the reversed transactions: {}".format(amount))

No. of reversed transactions: 6064
Total amount of the reversed transactions: 908034.4300000026


Almost 1% of the entire transaction amount of the dataset is reversed transactions

### [6]

### Final Feature Engineering

In [44]:
FirstPurchase = []
Vendor = []
accntnum=new_df['accountNumber'].unique().tolist()
for x in accntnum:
    temp_df = new_df[new_df["accountNumber"]==x].reset_index()
    for i in range(len(temp_df)):
        if temp_df.merchantName[i] in FirstPurchase:
            Vendor.append(False)
        else:
            FirstPurchase.append(temp_df.merchantName[i])
            Vendor.append(True)
    

##### Descripition

The above performs whether the customer has ever bought anything from that vendor or not.

**Assumption:** The first transaction is always a new transaction, if there is another transaction from the same vendor later on from the customer, it will append a False value.

##### Input Parameters:

Sliced Dataframe

##### Output:

A list of vendors whether they are first purchased vendors or not.

**Suggestion:**

Forgot writing the code  
    
    temp_df = new_df[new_df["accountNumber"]==x]
    
    as 
    
    temp_df = new_df[new_df["accountNumber"]==x].reset_index()
    
Took one hour for troubleshooting. So if you are worrying why the indexes are not being assigned properly, then this is the reason why. 


In [45]:
new_df["FirstVendorPurchase"] = Vendor

#### [6.1]
#### Marking the Multiswipe and Unknown transactions

In [34]:
for i in range(len(new_df)):
    if(new_df.multiswipeMinutes[i] <= 2 and new_df.multiswipeMinutes[i] >0):
        if(new_df.transactionType[i]=="REVERSAL"): 
            if(new_df.transactionType[i]=="" or "PURCHASE"):
                new_df.transactionType[i] = "MULTISWIPE"

In [35]:
for i in range(len(new_df)):
    if(new_df.transactionType[i]==""):
            new_df.transactionType[i] = "UNKNOWN"

#### [6.2]
#### Creating the columns sameCountry which has a bool value if the purchase was made in the same country or not

In [36]:
new_df["sameCountry"] = new_df["merchantCountryCode"]==new_df["acqCountry"]

##### Dropping a few unnecessary Columns

In [37]:
new_df = new_df.drop(["acqCountry", "merchantCountryCode", "merchantName"],1)

Filling the null rows of the posEntryMode and posConditionCode with a variable '00'.

**Explanation:**

I chose them to be strings as opposed to be integers as my assumption of them being the card being inserted into the machine. This will help when encoding  the stuff and the classifier will not assume random weights

In [40]:
for i in range(len(new_df)):
    if(new_df.posEntryMode[i]==""):
            new_df.posEntryMode[i] = "00"
            
    if(new_df.posConditionCode[i]==""):
            new_df.posConditionCode[i] = "00"

In [41]:
account_number = new_df["accountNumber"]

#### [6.3]

##### Dropping the final columns and making a pickle of the dataset for further use.

In [42]:
new_df = new_df.drop(["accountNumber", "transactionDateTime"], 1)

In [44]:
new_df["accountNumber"] = account_Number

In [45]:
new_df.to_pickle("./final")

## [7] Final Thoughts and Conclusions:

- Using my methods, there are 12925 multiswipe transactions and 6064 reversed transactions.
- Total amount of the reversed transactions: 908034.43, which is almost 1% of the entire transactionAmount in the dataset.
- Total amount of the multiswiped transactions: 1878098.28 which is almost 2% of the entire transactionAmount in the dataset.

